# Introduction
This notebook contains the 4 (+2 bonus) agents used in the paper "Using Hippocampal Replay to Consolidate Experiences in Memory-Augmented Reinforcement Learning"
- Random
- TD (bonus)
- Q-Learning (bonus)
- Go-Explore
- Go-Explore-Count
- Explore-Count

It contains the 2 discrete state environments used
- Unwalled Maze
- Walled Maze

It contains 2 bonus discrete state environments:
- Towers of Hanoi
- Nim

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy
from collections import defaultdict

# Agent Definition

In [2]:
# this is the memory for the agents which need them
memory = defaultdict(lambda: 0)

## Agent 1: Random Agent

In [3]:
def RandomAgent(env, **kwargs):
    return env.sample()

## Agent 2: TD Agent

TD-error: $\delta_t = r_{t} + \gamma \max_{a\in A, a: s_t \rightarrow s_{t+1}}V(s_{t+1}) - V(s_t)$

Value update: $V(s_t) \leftarrow V(s_t) + \alpha\delta_t$

In [4]:
def TDAgent(env, **kwargs):
    eps = kwargs.get('eps', 1)
    GAMMA = 0.99
    ALPHA = 1
    
    curstate = env.staterep()
    
    if env.reward == 1:
        memory[curstate] = 1
        return

    validmoves = env.getvalidmoves()
    bestmove = None
    bestvalue = -1
    
    # choose best move
    for move in validmoves:
        newenv = copy.deepcopy(env)
        newenv.step(move)
        nextstate = newenv.staterep()
            
        curvalue = memory[nextstate] 
        if curvalue > bestvalue:
            bestvalue = curvalue
            bestmove = move
    
    # if epsilon, then choose randomly
    if eps > np.random.rand():
        bestmove = env.sample()
        
    # update current state value with optimal one-step lookahead estimate
    td_error = env.reward + GAMMA*bestvalue - memory[curstate]
    memory[curstate] = memory[curstate] + ALPHA*td_error
    
    return bestmove

## Agent 3: Q-Learning Agent

TD-error: $\delta_t = r_t + \gamma\max_{a\in A}Q(s_{t+1},a) - Q(s_t, a_t)$

Q-learning update: $Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha\delta_t$

In [5]:
def QAgent(env, **kwargs):
    eps = kwargs.get('eps', 1)
    
    GAMMA = 0.99
    ALPHA = 1
    
    curstate = env.staterep()
    
    if env.reward == 1:
        for move in env.getvalidmoves():
            memory[(curstate, move)] = 1
        return

    validmoves = env.getvalidmoves()
    bestmove = None
    bestvalue = -1
    
    # if epsilon, then choose randomly
    if eps > np.random.rand():
        bestmove = env.sample()
    # else choose best move
    else:
        for move in validmoves:
            curvalue = memory[(curstate, move)] 
            if curvalue > bestvalue:
                bestvalue = curvalue
                bestmove = move
        
    # do a one-step in the next direction
    newenv = copy.deepcopy(env)
    newenv.step(bestmove)
    nextstate = newenv.staterep()
    td_error = env.reward + GAMMA*np.max([memory[(nextstate, move)] for move in newenv.getvalidmoves()]) - memory[(curstate, bestmove)]
        
    # update the Q-function
    memory[(curstate, bestmove)] = memory[(curstate, bestmove)] + ALPHA*td_error
    
    return bestmove

## Agent 4: Go-Explore

In [6]:
def reward_formula(reward = 0, intrinsic = 0, moves = 0, numselected = 0, numvisits = 0, eps = 1e-20):
    return 10*reward + 10*intrinsic + 1*np.sqrt(moves) - 10*np.sqrt(numselected+numvisits)
    # return 10*reward + 1*np.sqrt(moves) - 10*np.sqrt(numselected+numvisits)

In [7]:
''' Chooses the next best memory state to go to '''
def ChooseState(env):
    bestvalue = -1e20
    bestkey = None
    
    # choose the best memory based on heuristics
    for key in memory:
        # do not choose final state as there is nothing left to explore
        if memory[key]['reward'] == 1: 
            continue
        
        curmem = memory[key]
        reward = curmem['reward']
        intrinsic = curmem['intrinsic']
        moves = curmem['moves']
        numselected = curmem.get('numselected', 0)
        numvisits = curmem['numvisits']
        
        value = reward_formula(reward = reward, intrinsic = intrinsic, moves = moves, numselected = numselected, numvisits = numvisits)
        if value > bestvalue:
            bestvalue = value
            bestkey = key
    
    # generate the trajectory to get the environment state
    actionhistory = []
    
    for move in memory[bestkey]['actionhistory']:
        env.step(move)
        actionhistory.append(move)
        
    # increment the selection visit count
    if 'numselected' in memory[bestkey]:
        memory[bestkey]['numselected'] = memory[bestkey]['numselected'] + 1
    
    return (actionhistory, copy.deepcopy(env))
        

In [8]:
''' Chooses the best move based on memory and intrinsic rewards '''
def GoExplore(env, **kwargs):
    
    # if completed, no need to continue to next move selection
    if env.done:
        return
    
    intrinsic_fn = kwargs.get('intrinsic_fn', None)
    replay = kwargs.get('replay', False)
    getbestmove = kwargs.get('getbestmove', False)
    actionhistory = kwargs.get('actionhistory', [])
    
    # if no intrinsic guiding value, then do without intrinsic motivation
    if intrinsic_fn is not None:
        intrinsic_value = intrinsic_fn(env)
    else:
        intrinsic_value = 0
        
    curmoves = env.numsteps
    curreward = env.reward

    curstate = env.staterep()
        
    # if this state is not present in memory (should only happen for start state), add it in
    if curstate not in memory:
        memory[curstate] = {'reward': curreward, 'intrinsic': intrinsic_value, 'moves': curmoves, 'numvisits': 0, 'numselected': 0, 'actionhistory': actionhistory+[]}
        
    curmemory = memory[curstate]
    
    # only increment memory if not doing replay
    if replay:
        curmemory['numvisits'] = 0
        curmemory['numselected'] = 0
    else:
        curmemory['numvisits'] = curmemory['numvisits'] + 1

    # if not completed, continue to select next move
    validmoves = env.getvalidmoves()
    
    # if no valid moves, no need to continue to next move selection
    if validmoves == []:
        return
    
    bestmove = None
    bestvalue = -1e20
    bestintrinsic = -1e20
    
    # choose best move
    for move in validmoves:
        newenv = copy.deepcopy(env)
        newenv.step(move)
        
        nextmoves = newenv.numsteps
        nextreward = newenv.reward
        nextmemory = None
        
        nextstate = newenv.staterep()
        
        if nextstate in memory:
            nextmemory = memory[nextstate] 
            # update the nextmemory if agent has a better reward
            if nextreward > nextmemory['reward']:
                nextmemory['reward'] = nextreward
                nextmemory['moves'] = curmoves + 1
                nextmemory['numvisits'] = 0
                nextmemory['numselected'] = 0
                nextmemory['actionhistory'] = actionhistory+[move]
                
            # update the nextmemory if agent has similar reward but fewer number of moves
            elif nextreward == nextmemory['reward'] and nextmemory['moves'] > curmoves + 1:
                nextmemory['moves'] = curmoves + 1
                nextmemory['numvisits'] = 0
                nextmemory['numselected'] = 0
                nextmemory['actionhistory'] = actionhistory+[move]
                
        # start a new memory if this is a new state
        else:
            # if no intrinsic guiding value, then do without intrinsic motivation
            if intrinsic_fn is not None:
                next_intrinsic_value = intrinsic_fn(newenv)
            else:
                next_intrinsic_value = 0
            memory[nextstate] = {'reward': nextreward, 'intrinsic': next_intrinsic_value, 'moves': curmoves + 1, 'numvisits': 0, 'numselected': 0, 'actionhistory': actionhistory+[move]}
            nextmemory = memory[nextstate]
                
        # best intrinsic is the highest intrinsic value of all 1-step connections
        bestintrinsic = max(bestintrinsic, nextmemory['intrinsic'])
        
        # determine the next square to visit via a heuristic
        reward = nextmemory['reward'] 
        moves = nextmemory['moves'] 
        numvisits = nextmemory['numvisits']
        intrinsic = nextmemory['intrinsic']
        
        totalvalue = reward_formula(reward = reward, intrinsic = intrinsic, moves = moves, numselected = 0, numvisits = numvisits)
      
        if totalvalue > bestvalue or bestvalue is None:
            bestvalue = totalvalue
            bestmove = move
            
    # if replay:
    # update the one-step lookahead for intrinsic value
    curmemory['intrinsic'] = bestintrinsic*0.99

    if getbestmove:
        return bestmove
    else:
        return np.random.choice(validmoves)

In [9]:
''' Chooses Best Move '''
def GoExploreCount(env, **kwargs):
    return GoExplore(env, getbestmove = True, **kwargs)

## Agent 5: Count Agent

In [10]:
''' Chooses the best move based on memory and intrinsic rewards '''
def CountAgent(env, **kwargs):
    
    # if completed, no need to continue to next move selection
    if env.done:
        return
    
    intrinsic_fn = kwargs.get('intrinsic_fn', None)
    replay = kwargs.get('replay', False)
    actionhistory = kwargs.get('actionhistory', [])
    
    # if no intrinsic guiding value, then do without intrinsic motivation
    if intrinsic_fn is not None:
        intrinsic_value = intrinsic_fn(env)
    else:
        intrinsic_value = 0
        
    curmoves = env.numsteps
    curreward = env.reward

    curstate = env.staterep()

    # if this state is not present in memory (should only happen for start state), add it in
    if curstate not in memory:
        memory[curstate] = {'reward': curreward, 'intrinsic': intrinsic_value, 'moves': curmoves, 'numvisits': 0, 'actionhistory': actionhistory+[]}
        
    curmemory = memory[curstate]
    
    # only increment memory if not doing replay
    if replay:
        curmemory['numvisits'] = 0
        curmemory['numselected'] = 0
    else:
        curmemory['numvisits'] = curmemory['numvisits'] + 1

    # if not completed, continue to select next move
    validmoves = env.getvalidmoves()
    
    # if no valid moves, no need to continue to next move selection
    if validmoves == []:
        return
    
    bestmove = None
    bestvalue = -1e20
    bestintrinsic = -1e20
    
    # choose best move
    for move in validmoves:
        newenv = copy.deepcopy(env)
        newenv.step(move)
        
        nextmoves = newenv.numsteps
        nextreward = newenv.reward
        nextmemory = None
        
        nextstate = newenv.staterep()

        if nextstate in memory:
            nextmemory = memory[nextstate] 
            # update the nextmemory if agent has a better reward
            if nextreward > nextmemory['reward']:
                nextmemory['reward'] = nextreward
                nextmemory['moves'] = curmoves + 1
                nextmemory['numvisits'] = 0
                nextmemory['actionhistory'] = actionhistory+[move]
                
            # update the nextmemory if agent has similar reward but fewer number of moves
            elif nextreward == nextmemory['reward'] and nextmemory['moves'] > curmoves + 1:
                nextmemory['moves'] = curmoves + 1
                nextmemory['numvisits'] = 0
                nextmemory['actionhistory'] = actionhistory+[move]
                
        # start a new memory if this is a new state
        else:
            # if no intrinsic guiding value, then do without intrinsic motivation
            if intrinsic_fn is not None:
                next_intrinsic_value = intrinsic_fn(newenv)
            else:
                next_intrinsic_value = 0
            memory[nextstate] = {'reward': nextreward, 'intrinsic': next_intrinsic_value, 'moves': curmoves + 1, 'numvisits': 0, 'actionhistory': actionhistory+[move]}
            nextmemory = memory[nextstate]
            
        # best intrinsic is the highest intrinsic value of all 1-step connections
        bestintrinsic = max(bestintrinsic, nextmemory['intrinsic'])
        
        # determine the next square to visit via a heuristic
        reward = nextmemory['reward'] 
        moves = nextmemory['moves'] 
        numvisits = nextmemory['numvisits']
        intrinsic = nextmemory['intrinsic']
        
        totalvalue = reward_formula(reward = reward, intrinsic = intrinsic, moves = moves, numselected = 0, numvisits = numvisits)
        
        if totalvalue > bestvalue:
            bestvalue = totalvalue
            bestmove = move
            
        # update the one-step lookahead for intrinsic value
        curmemory['intrinsic'] = bestintrinsic*0.99
    
    return bestmove

# Helper Functions
These functions help to perform hippocampal replay, and evaluation of agent on the environment.
- MemoryReplay: Implements hippocampal replay
- Game: Plays an environment for a single run (episode)
- MultipleGame: Plays an environment for 100 runs

## Memory Replay

In [11]:
''' Performs memory replay '''
def MemoryReplay(env, bestactionhistory = [], agent = RandomAgent, maxsteps = 500, seed = None, **kwargs):
    statelist = []
    kwargs['replay']=True
    
    # do forward replay to consolidate states
    for move in bestactionhistory:     
        statelist.append(copy.deepcopy(env))
        env.step(move)     
    
    # # do backward replay to update memory
    for env in statelist[::-1]:    
        agent(env, **kwargs)

## A Single Game

In [12]:
''' Plays 1 game '''
def Game(env, agent = RandomAgent, actionhistory = [], maxsteps = 500, seed = None, verbose = True, **kwargs):
    
    if seed is not None:
        np.random.seed(seed)
    else:
        np.random.seed(0)
    while not env.done and env.numsteps < maxsteps:  
        kwargs['actionhistory'] = actionhistory
        move = agent(env, **kwargs)
        env.step(move)
        actionhistory.append(move)

    kwargs['actionhistory'] = actionhistory
    # to update final state for RL agents
    agent(env, **kwargs)
    
    if verbose:
        print(env.done, env.reward, env.numsteps)

    return env.done, env.reward, env.numsteps

## Multiple Games

In [13]:
''' Plays multiple games '''
def MultiGame(env, numtries = 100, hippocampal_replay = True, latex = False, **kwargs):
    solvedcount = 0
    stephistory = []
    bestmemory = 0
    beststeps = 1000000
    firstsolve = None
    tries = numtries
    solved = False
    
    for i in range(tries):
        # choose a new state for GoExplore or GoExploreCount
        if kwargs['agent'] in [GoExplore, GoExploreCount] and i > 0:
            actionhistory, nextenv = ChooseState(env = copy.deepcopy(env))
            done, reward, steps = Game(seed = i, env = copy.deepcopy(nextenv), actionhistory = actionhistory+[], **kwargs)
        else:
            done, reward, steps = Game(seed = i, env = copy.deepcopy(env), actionhistory = [], **kwargs)
        if reward == 1:
            solvedcount += 1
            stephistory.append(steps)
            
            # if first solve, note how much memory is used
            if solvedcount == 1:
                bestmemory = len(memory)
                firstsolve = i+1
                
            if hippocampal_replay:
                # hippocampal replay only for goexplore or intrinsic agent
                if kwargs['agent'] in [GoExplore, GoExploreCount, CountAgent]:
                    actionhistory = None
                    for key, value in memory.items():
                        if memory[key]['reward'] == 1:
                            actionhistory = memory[key]['actionhistory']

                    # MemoryReplay to improve chance of optimal path being followed
                    if actionhistory is not None:
                        MemoryReplay(env = copy.deepcopy(env), bestactionhistory = actionhistory, **kwargs)
    name = kwargs['agent'].__name__
    if name == 'RandomAgent': 
        name = 'Random'
        bestmemory = '-'
    if name == 'QAgent': name = 'Q-Learning'
    if name == 'TDAgent': name = 'TD-Learning'
    if name == 'GoExplore': name = 'Go-Explore'
    if name == 'GoExploreCount': name = 'Go-Explore-Count'
    if name == 'CountAgent': name = 'Explore-Count'

    if kwargs['agent'] == QAgent or kwargs['agent'] == TDAgent:
        if kwargs.get('eps', 1) == 0:
            name += ' (Test)'
        else:
            name += ' (Train)'
            
    if kwargs.get('intrinsic_fn', None) is not None:
        name += ' GDIR'

    if 'Explore' in name and hippocampal_replay:
        name += '-HR'

    if latex:
        if solvedcount == 0:
            print(f"{name} & {solvedcount}/{tries} & - & - & - & - & - \\\\")
        else:
            print(f"{name} & {solvedcount}/{tries} & {firstsolve} & {bestmemory} & {sum(stephistory)/len(stephistory):.1f} & {min(stephistory):.1f} & {max(stephistory):.1f} \\\\")
    else:
        if solvedcount == 0:
            print(f'Agent: {name}, No solves at all, First Solve Memory: {bestmemory}, Total Memory: {len(memory)}')
        else:
            print(f'Agent: {name}, Solve rate: {solvedcount}/{tries} ({solvedcount/tries*100:.1f}%), First Solve: {firstsolve}, First Solve Memory: {bestmemory}, Steps: Avg {sum(stephistory)/len(stephistory):.1f}, Min {min(stephistory):.1f}, Max {max(stephistory):.1f}')

# Discrete Environments 1 & 2 - Maze Environment (Unwalled, Walled)

In [14]:
class MazeEnv:
    def __init__(self, height=20, width=20, numbricks = 10, grid = None, doorpos = None, agentpos = None, randomseed = None):
        self.height = height
        self.width = width
        self.doorpos = doorpos
        self.agentpos = agentpos
        self.numbricks = numbricks
        self.randomseed = randomseed
        self.numsteps = 0
        self.done = False
        self.reward = 0
        if self.randomseed is not None:
            np.random.seed(self.randomseed)
        self.mapping = {0: '.', 1: 'X', 2: 'D', 3: '#'}
        
        # if grid not defined, do a random initialization of maze
        if grid is None:
            self.grid = np.zeros((self.height, self.width))
            
            # Step 1: get a door position that is valid
            if doorpos is None:
                self.doorpos = self.getvalidpos()
            else:
                self.doorpos = doorpos
            self.grid[self.doorpos] = 2

            # Step 2: get a start position that is valid
            if agentpos is None:
                self.agentpos = self.getvalidpos()
            else:
                self.agentpos = agentpos
            self.grid[self.agentpos] = 1
            
            # Step 3: fill in the bricks
            for i in range(self.numbricks):
                self.grid[self.getvalidpos()] = 3
                
        # if grid predefined, get the parameters from there instead
        else:
            self.grid = grid
            self.height, self.width = self.grid.shape
            
            lista, listb = np.where(self.grid == 2)
            if len(lista) == 0 or len(listb) == 0:
                self.doorpos = self.getvalidpos()
            else:
                self.doorpos = (lista[0], listb[0])
            self.grid[self.doorpos] = 2
            
            lista, listb = np.where(self.grid == 1)
            if len(lista) == 0 or len(listb) == 0:
                self.agentpos = self.getvalidpos()
            else:
                self.agentpos = (lista[0], listb[0])
            self.grid[self.agentpos] = 1
            
        # some variables to reset the environment
        self.startgrid = self.grid.copy()
        self.startagentpos = self.agentpos
        self.startdoorpos = self.doorpos
            
    def reset(self):
        self.grid = self.startgrid.copy()
        self.agentpos = self.startagentpos
        self.doorpos = self.startdoorpos
        self.done = False
        self.reward = 0
        self.numsteps = 0
        if self.randomseed is not None:
            np.random.seed(self.randomseed)
            
    # gets state representation
    def staterep(self):
        return str(self.agentpos)
            
    # gets a valid position
    def getvalidpos(self):
        validpos = []
        for i in range(self.height):
            for j in range(self.width):
                if self.grid[i,j] == 0:
                    validpos.append((i,j))
        return validpos[np.random.randint(len(validpos))]
        
    # checks if a position is valid that is not out of the grid and not occupied
    def isvalid(self, pos, allowdoor = False):
        if pos == None or len(pos)!=2:
            return False
        height, width = pos
        if height < 0 or height >= self.height or width < 0 or width >= self.width:
            return False
        if allowdoor and self.grid[height,width] == 2:
            return True
        if self.grid[height,width] == 0:
            return True
        return False
    
    def step(self, move):
        validmoves = self.getvalidmoves()
        # randomly sample a move if not in validmoves
        if move not in validmoves:
            move = validmoves[np.random.randint(len(validmoves))]
        self.numsteps += 1
        self.grid[self.agentpos] = 0
        self.agentpos = self.movedir(self.agentpos, move)
        if self.agentpos == self.doorpos:
            self.done = True
            self.reward = 1
        self.grid[self.agentpos] = 1
    
    def movedir(self, pos, d):
        if pos == None or len(pos)!=2:
            return False
        height, width = pos
        if d=='left':
            return (height, width-1)
        elif d=='right':
            return (height, width+1)
        elif d=='up':
            return (height-1, width)
        elif d=='down':
            return (height+1, width)
    
    def getvalidmoves(self):
        validmoves = []
        for move in ['left', 'right', 'up', 'down']:
            if self.isvalid(self.movedir(self.agentpos, move), allowdoor = True):
                validmoves.append(move)
        return validmoves
    
    def sample(self):
        return np.random.choice(self.getvalidmoves())
    
    def print(self):
        for i in range(self.height):
            for j in range(self.width):
                print(self.mapping[self.grid[i,j]], end = '')
            print()

In [15]:
def Manhattan(env):
    ''' Calculates the Manhattan distance between the agent and the door '''
    pointA = env.agentpos
    pointB = env.doorpos
    return -(abs(pointA[0]-pointB[0])+abs(pointA[1]-pointB[1]))/(env.width+env.height-2)

## Unwalled maze (10x10)

In [16]:
# This is how the maze looks like
height, width = 10, 10
env = MazeEnv(height = height, width = width, agentpos = (0, 0), doorpos = (height-1, width-1), randomseed = 1, numbricks = height*width//10)
env.print()

X.#...#...
#..#......
#.........
........#.
..........
..........
.........#
.....#....
#.....#...
.........D


In [17]:
latex = False # this is to determine if output is latex friendly

# for agent in [RandomAgent, TDAgent, QAgent]:
for agent in [RandomAgent]:
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, verbose = False)
    
    # for QAgent and TDAgent, we include the final number of solves with deterministic transition
    if agent == QAgent or agent == TDAgent:
        MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = env.height*env.width, verbose = False, eps = 0)

for agent in [GoExplore, GoExploreCount, CountAgent]:
    # without hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, hippocampal_replay = False, verbose = False, intrinsic_fn = None)
    # with hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, verbose = False, intrinsic_fn = None)
    # intrinsic reward without hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, hippocampal_replay = False, verbose = False, intrinsic_fn = Manhattan)
    # intrinsic reward with hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, verbose = False, intrinsic_fn = Manhattan)

Agent: Random, Solve rate: 8/100 (8.0%), First Solve: 8, First Solve Memory: -, Steps: Avg 76.8, Min 54.0, Max 100.0
Agent: Go-Explore, No solves at all, First Solve Memory: 0, Total Memory: 87
Agent: Go-Explore-HR, No solves at all, First Solve Memory: 0, Total Memory: 87
Agent: Go-Explore GDIR, Solve rate: 5/100 (5.0%), First Solve: 12, First Solve Memory: 59, Steps: Avg 88.0, Min 64.0, Max 100.0
Agent: Go-Explore GDIR-HR, Solve rate: 15/100 (15.0%), First Solve: 12, First Solve Memory: 59, Steps: Avg 80.0, Min 50.0, Max 96.0
Agent: Go-Explore-Count, Solve rate: 89/100 (89.0%), First Solve: 1, First Solve Memory: 71, Steps: Avg 41.7, Min 18.0, Max 94.0
Agent: Go-Explore-Count-HR, Solve rate: 99/100 (99.0%), First Solve: 1, First Solve Memory: 71, Steps: Avg 62.3, Min 62.0, Max 94.0
Agent: Go-Explore-Count GDIR, Solve rate: 90/100 (90.0%), First Solve: 1, First Solve Memory: 39, Steps: Avg 42.8, Min 18.0, Max 100.0
Agent: Go-Explore-Count GDIR-HR, Solve rate: 100/100 (100.0%), First S

## Unwalled maze (20x20)

In [18]:
# This is how the maze looks like
height, width = 20, 20
env = MazeEnv(height = height, width = width, agentpos = (0, 0), doorpos = (height-1, width-1), randomseed = 1, numbricks = height*width//10)
env.print()

X.#.....#...........
....#.........#...#.
............#.......
...........#.##.....
............#.......
....................
.....#......##..#...
........#..#...#....
...#................
.....#..............
......#.............
......##.......#.#..
......#.........#..#
....#.#.............
........#.#.#.......
....................
..................#.
.##.#.#.............
.............#.#....
..................#D


In [19]:
latex = False # this is to determine if output is latex friendly

# for agent in [RandomAgent, TDAgent, QAgent]:
for agent in [RandomAgent]:
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, verbose = False)
    
    # for QAgent and TDAgent, we include the final number of solves with deterministic transition
    if agent == QAgent or agent == TDAgent:
        MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = env.height*env.width, verbose = False, eps = 0)

for agent in [GoExplore, GoExploreCount, CountAgent]:
    # without hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, hippocampal_replay = False, verbose = False, intrinsic_fn = None)
    # with hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, verbose = False, intrinsic_fn = None)
    # intrinsic reward without hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, hippocampal_replay = False, verbose = False, intrinsic_fn = Manhattan)
    # intrinsic reward with hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, verbose = False, intrinsic_fn = Manhattan)

Agent: Random, No solves at all, First Solve Memory: -, Total Memory: 0
Agent: Go-Explore, No solves at all, First Solve Memory: 0, Total Memory: 259
Agent: Go-Explore-HR, No solves at all, First Solve Memory: 0, Total Memory: 259
Agent: Go-Explore GDIR, No solves at all, First Solve Memory: 0, Total Memory: 242
Agent: Go-Explore GDIR-HR, No solves at all, First Solve Memory: 0, Total Memory: 242
Agent: Go-Explore-Count, Solve rate: 75/100 (75.0%), First Solve: 2, First Solve Memory: 359, Steps: Avg 176.5, Min 78.0, Max 398.0
Agent: Go-Explore-Count-HR, Solve rate: 99/100 (99.0%), First Solve: 2, First Solve Memory: 359, Steps: Avg 342.0, Min 342.0, Max 346.0
Agent: Go-Explore-Count GDIR, Solve rate: 29/100 (29.0%), First Solve: 1, First Solve Memory: 85, Steps: Avg 200.2, Min 42.0, Max 398.0
Agent: Go-Explore-Count GDIR-HR, Solve rate: 100/100 (100.0%), First Solve: 1, First Solve Memory: 85, Steps: Avg 42.0, Min 42.0, Max 42.0
Agent: Explore-Count, Solve rate: 85/100 (85.0%), First S

## Unwalled maze (100x100)

In [20]:
# This is how the maze looks like
height, width = 100, 100
env = MazeEnv(height = height, width = width, agentpos = (0, 0), doorpos = (height-1, width-1), randomseed = 1, numbricks = height*width//10)
env.print()

X..#............#...#.#...........##.........#..........#.........#........##........#........#...#.
.......#.#...#.........#.................#...#...#...#..###...................#....#..#....#......#.
....................................#...#.......#...............#...........#.....#.......#......#..
...............#............#..#................#...........#..............................#.#.#....
.......#...............#..........#.................................#..............#.....#..#.......
#.........#.......#.#...........#.....#.#.....#..........#.......#..................................
..........#...........#..#....................#.........#.#.............................#....#......
.#...........#.##.........#.....................#........##.......#....................#....#...#...
.....#............####..............#........#.#..##......#.......#.........................#.......
....#..#.#..........#.......................#............#........#..................##....

In [21]:
latex = False # this is to determine if output is latex friendly

# for agent in [RandomAgent, TDAgent, QAgent]:
for agent in [RandomAgent]:
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, verbose = False)
    
    # for QAgent and TDAgent, we include the final number of solves with deterministic transition
    if agent == QAgent or agent == TDAgent:
        MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = env.height*env.width, verbose = False, eps = 0)

for agent in [GoExplore, GoExploreCount, CountAgent]:
    # without hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, hippocampal_replay = False, verbose = False, intrinsic_fn = None)
    # with hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, verbose = False, intrinsic_fn = None)
    # intrinsic reward without hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, hippocampal_replay = False, verbose = False, intrinsic_fn = Manhattan)
    # intrinsic reward with hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, verbose = False, intrinsic_fn = Manhattan)

Agent: Random, Solve rate: 1/100 (1.0%), First Solve: 15, First Solve Memory: -, Steps: Avg 9980.0, Min 9980.0, Max 9980.0
Agent: Go-Explore, No solves at all, First Solve Memory: 0, Total Memory: 3116
Agent: Go-Explore-HR, No solves at all, First Solve Memory: 0, Total Memory: 3116
Agent: Go-Explore GDIR, No solves at all, First Solve Memory: 0, Total Memory: 3111
Agent: Go-Explore GDIR-HR, No solves at all, First Solve Memory: 0, Total Memory: 3111
Agent: Go-Explore-Count, Solve rate: 78/100 (78.0%), First Solve: 1, First Solve Memory: 7597, Steps: Avg 5533.1, Min 4312.0, Max 9498.0
Agent: Go-Explore-Count-HR, Solve rate: 100/100 (100.0%), First Solve: 1, First Solve Memory: 7597, Steps: Avg 6003.3, Min 5984.0, Max 7854.0
Agent: Go-Explore-Count GDIR, Solve rate: 2/100 (2.0%), First Solve: 1, First Solve Memory: 499, Steps: Avg 231.0, Min 230.0, Max 232.0
Agent: Go-Explore-Count GDIR-HR, Solve rate: 100/100 (100.0%), First Solve: 1, First Solve Memory: 499, Steps: Avg 230.0, Min 230.

## Walled maze (10x10)

In [22]:
# create game environment
size = 10
grid = np.zeros((size,size))
maxheight, maxwidth = grid.shape
grid[:, maxwidth//2-1] = 3
grid[maxheight//2,:] = 3
grid[1:maxheight, maxwidth-2] = 3
grid[maxheight//2, maxwidth//4] = 0
grid[maxheight//2, 3*maxwidth//4-1] = 0
grid[3*maxheight//4, maxwidth//2-1] = 0
grid[maxheight//2, maxwidth-1] = 0
grid[0, 0] = 1
grid[maxheight-1, maxwidth-1] = 2
env = MazeEnv(grid = grid)
env.print()

X...#.....
....#...#.
....#...#.
....#...#.
....#...#.
##.###.##.
....#...#.
........#.
....#...#.
....#...#D


In [23]:
latex = False # this is to determine if output is latex friendly

# for agent in [RandomAgent, TDAgent, QAgent]:
for agent in [RandomAgent]:
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, verbose = False)
    
    # for QAgent and TDAgent, we include the final number of solves with deterministic transition
    if agent == QAgent or agent == TDAgent:
        MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = env.height*env.width, verbose = False, eps = 0)

for agent in [GoExplore, GoExploreCount, CountAgent]:
    # without hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, hippocampal_replay = False, verbose = False, intrinsic_fn = None)
    # with hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, verbose = False, intrinsic_fn = None)
    # intrinsic reward without hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, hippocampal_replay = False, verbose = False, intrinsic_fn = Manhattan)
    # intrinsic reward with hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, verbose = False, intrinsic_fn = Manhattan)

Agent: Random, No solves at all, First Solve Memory: -, Total Memory: 0
Agent: Go-Explore, No solves at all, First Solve Memory: 0, Total Memory: 77
Agent: Go-Explore-HR, No solves at all, First Solve Memory: 0, Total Memory: 77
Agent: Go-Explore GDIR, No solves at all, First Solve Memory: 0, Total Memory: 77
Agent: Go-Explore GDIR-HR, No solves at all, First Solve Memory: 0, Total Memory: 77
Agent: Go-Explore-Count, Solve rate: 85/100 (85.0%), First Solve: 1, First Solve Memory: 74, Steps: Avg 57.1, Min 32.0, Max 100.0
Agent: Go-Explore-Count-HR, Solve rate: 100/100 (100.0%), First Solve: 1, First Solve Memory: 74, Steps: Avg 64.0, Min 64.0, Max 64.0
Agent: Go-Explore-Count GDIR, Solve rate: 71/100 (71.0%), First Solve: 1, First Solve Memory: 62, Steps: Avg 50.7, Min 32.0, Max 100.0
Agent: Go-Explore-Count GDIR-HR, Solve rate: 100/100 (100.0%), First Solve: 1, First Solve Memory: 62, Steps: Avg 60.0, Min 60.0, Max 60.0
Agent: Explore-Count, Solve rate: 71/100 (71.0%), First Solve: 1, 

## Walled maze (20x20)

In [24]:
# create game environment
size = 20
grid = np.zeros((size,size))
maxheight, maxwidth = grid.shape
grid[:, maxwidth//2-1] = 3
grid[maxheight//2,:] = 3
grid[1:maxheight, maxwidth-2] = 3
grid[maxheight//2, maxwidth//4] = 0
grid[maxheight//2, 3*maxwidth//4-1] = 0
grid[3*maxheight//4, maxwidth//2-1] = 0
grid[maxheight//2, maxwidth-1] = 0
grid[0, 0] = 1
grid[maxheight-1, maxwidth-1] = 2
env = MazeEnv(grid = grid)
env.print()

X........#..........
.........#........#.
.........#........#.
.........#........#.
.........#........#.
.........#........#.
.........#........#.
.........#........#.
.........#........#.
.........#........#.
#####.########.####.
.........#........#.
.........#........#.
.........#........#.
.........#........#.
..................#.
.........#........#.
.........#........#.
.........#........#.
.........#........#D


In [25]:
latex = False # this is to determine if output is latex friendly

# for agent in [RandomAgent, TDAgent, QAgent]:
for agent in [RandomAgent]:
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, verbose = False)
    
    # for QAgent and TDAgent, we include the final number of solves with deterministic transition
    if agent == QAgent or agent == TDAgent:
        MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = env.height*env.width, verbose = False, eps = 0)

for agent in [GoExplore, GoExploreCount, CountAgent]:
    # without hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, hippocampal_replay = False, verbose = False, intrinsic_fn = None)
    # with hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, verbose = False, intrinsic_fn = None)
    # intrinsic reward without hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, hippocampal_replay = False, verbose = False, intrinsic_fn = Manhattan)
    # intrinsic reward with hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, verbose = False, intrinsic_fn = Manhattan)

Agent: Random, No solves at all, First Solve Memory: -, Total Memory: 0
Agent: Go-Explore, No solves at all, First Solve Memory: 0, Total Memory: 322
Agent: Go-Explore-HR, No solves at all, First Solve Memory: 0, Total Memory: 322
Agent: Go-Explore GDIR, Solve rate: 4/100 (4.0%), First Solve: 60, First Solve Memory: 347, Steps: Avg 398.0, Min 394.0, Max 400.0
Agent: Go-Explore GDIR-HR, Solve rate: 10/100 (10.0%), First Solve: 60, First Solve Memory: 347, Steps: Avg 396.8, Min 394.0, Max 400.0
Agent: Go-Explore-Count, Solve rate: 76/100 (76.0%), First Solve: 1, First Solve Memory: 312, Steps: Avg 211.2, Min 144.0, Max 398.0
Agent: Go-Explore-Count-HR, Solve rate: 100/100 (100.0%), First Solve: 1, First Solve Memory: 312, Steps: Avg 218.0, Min 218.0, Max 218.0
Agent: Go-Explore-Count GDIR, Solve rate: 58/100 (58.0%), First Solve: 1, First Solve Memory: 223, Steps: Avg 177.3, Min 86.0, Max 380.0
Agent: Go-Explore-Count GDIR-HR, Solve rate: 100/100 (100.0%), First Solve: 1, First Solve Mem

## Walled maze (100x100)

In [26]:
# create game environment
size = 100
grid = np.zeros((size,size))
maxheight, maxwidth = grid.shape
grid[:, maxwidth//2-1] = 3
grid[maxheight//2,:] = 3
grid[1:maxheight, maxwidth-2] = 3
grid[maxheight//2, maxwidth//4] = 0
grid[maxheight//2, 3*maxwidth//4-1] = 0
grid[3*maxheight//4, maxwidth//2-1] = 0
grid[maxheight//2, maxwidth-1] = 0
grid[0, 0] = 1
grid[maxheight-1, maxwidth-1] = 2
env = MazeEnv(grid = grid)
env.print()

X................................................#..................................................
.................................................#................................................#.
.................................................#................................................#.
.................................................#................................................#.
.................................................#................................................#.
.................................................#................................................#.
.................................................#................................................#.
.................................................#................................................#.
.................................................#................................................#.
.................................................#.........................................

In [27]:
latex = False # this is to determine if output is latex friendly

# for agent in [RandomAgent, TDAgent, QAgent]:
for agent in [RandomAgent]:
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, verbose = False)
    
    # for QAgent and TDAgent, we include the final number of solves with deterministic transition
    if agent == QAgent or agent == TDAgent:
        MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = env.height*env.width, verbose = False, eps = 0)

for agent in [GoExplore, GoExploreCount, CountAgent]:
    # without hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, hippocampal_replay = False, verbose = False, intrinsic_fn = None)
    # with hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, verbose = False, intrinsic_fn = None)
    # intrinsic reward without hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, hippocampal_replay = False, verbose = False, intrinsic_fn = Manhattan)
    # intrinsic reward with hippocampal replay
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, latex = latex, maxsteps = env.height*env.width, verbose = False, intrinsic_fn = Manhattan)

Agent: Random, No solves at all, First Solve Memory: -, Total Memory: 0
Agent: Go-Explore, No solves at all, First Solve Memory: 0, Total Memory: 2334
Agent: Go-Explore-HR, No solves at all, First Solve Memory: 0, Total Memory: 2334
Agent: Go-Explore GDIR, No solves at all, First Solve Memory: 0, Total Memory: 2520
Agent: Go-Explore GDIR-HR, No solves at all, First Solve Memory: 0, Total Memory: 2520
Agent: Go-Explore-Count, Solve rate: 100/100 (100.0%), First Solve: 1, First Solve Memory: 7552, Steps: Avg 4918.2, Min 4718.0, Max 6362.0
Agent: Go-Explore-Count-HR, Solve rate: 100/100 (100.0%), First Solve: 1, First Solve Memory: 7552, Steps: Avg 4912.0, Min 4912.0, Max 4912.0
Agent: Go-Explore-Count GDIR, Solve rate: 97/100 (97.0%), First Solve: 1, First Solve Memory: 5105, Steps: Avg 8712.3, Min 8474.0, Max 9996.0
Agent: Go-Explore-Count GDIR-HR, Solve rate: 100/100 (100.0%), First Solve: 1, First Solve Memory: 5105, Steps: Avg 8922.0, Min 8922.0, Max 8922.0
Agent: Explore-Count, Solv

# Discrete Environment 3: Narrow Action Pathway - Tower of Hanoi

- Transfer all plates from first pole to third pole
- Each turn can move the top plate from an existing pole to another pole
- Constraint: the plate that is transferred to the new pole must be smaller than the current topmost plate of that pole
- Reward: 1 if all plates transferred successfully, 0 if not

In [28]:
class HanoiEnv:
    def __init__(self, numplates = 3, grid = None):
        self.numplates = numplates
        self.numsteps = 0
        self.done = False
        self.reward = 0
        # if grid not defined, then set to original position
        if grid is None:
            self.grid = [list(range(1, self.numplates+1)), [], []]
                
        # if grid predefined, get the parameters from there instead
        else:
            self.grid = grid
            
        # some variables to reset the environment
        self.startgrid = self.grid.copy()
            
    def reset(self):
        self.grid = self.startgrid.copy()
        self.done = False
        self.reward = 0
        self.numsteps = 0
        
    # returns state representation
    def staterep(self):
        return str(self.grid)
            
    # gets a valid move
    def getvalidmoves(self):
        validmoves = []
        for num, (pole1, pole2) in enumerate([(0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1)]):
            # if nothing to transfer, skip move
            if len(self.grid[pole1]) == 0: continue
            # if pole to be transferred to is empty, accept move
            if len(self.grid[pole2]) == 0: validmoves.append(num)
            # if piece to be transferred is smaller than the topmost piece of new pole, accept it
            elif self.grid[pole1][0] < self.grid[pole2][0]: validmoves.append(num)    
            
        return validmoves
    
    def step(self, move):
        validmoves = self.getvalidmoves()
        # randomly sample a move if not in validmoves
        if move not in validmoves:
            move = validmoves[np.random.randint(len(validmoves))]
        self.numsteps += 1
        
        movechoices = [(0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1)]
        # do the move
        pole1, pole2 = movechoices[move]
        self.grid[pole2] = [self.grid[pole1][0]] + self.grid[pole2]
        self.grid[pole1] = self.grid[pole1][1:]
        
        # check for completion
        if self.grid[2] == list(range(1, self.numplates+1)):
            self.reward = 1
            self.done = True
    
    def sample(self):
        return np.random.choice(self.getvalidmoves())
    
    def print(self):
        print(self.grid[0], self.grid[1], self.grid[2])

In [29]:
''' Number of disks away from solution (neg) '''
def Disk(env):
    finalpole = env.grid[2][::-1]
    totalsum = 0
    for i in range(env.numplates):
        if i+1 > len(finalpole): break
        if env.numplates - i != finalpole[i]: break
        # totalsum += env.numplates - i
        totalsum += 1
    # return (totalsum - np.sum(list(range(env.numplates+1))))/np.sum(list(range(env.numplates+1)))
    return (totalsum - env.numplates)/env.numplates

In [30]:
env = HanoiEnv(numplates = 3)

In [31]:
for agent in [RandomAgent, TDAgent, QAgent]:
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = 2**(env.numplates+2), verbose = False)
    
    # for QAgent and TDAgent, we include the final number of solves with deterministic transition
    if agent == QAgent or agent == TDAgent:
        MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = 2**(env.numplates+2), verbose = False, eps = 0)
    
for agent in [GoExplore, GoExploreCount, CountAgent]:
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = 2**(env.numplates+2), verbose = False, intrinsic_fn = None)

    # for Intrinsic Agent, we also do the run with intrinsic motivation
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = 2**(env.numplates+2), verbose = False, intrinsic_fn = Disk)

Agent: Random, Solve rate: 10/100 (10.0%), First Solve: 8, First Solve Memory: -, Steps: Avg 25.4, Min 17.0, Max 32.0
Agent: TD-Learning (Train), Solve rate: 13/100 (13.0%), First Solve: 4, First Solve Memory: 21, Steps: Avg 24.9, Min 13.0, Max 32.0
Agent: TD-Learning (Test), Solve rate: 100/100 (100.0%), First Solve: 1, First Solve Memory: 27, Steps: Avg 7.0, Min 7.0, Max 7.0
Agent: Q-Learning (Train), Solve rate: 13/100 (13.0%), First Solve: 4, First Solve Memory: 46, Steps: Avg 24.9, Min 13.0, Max 32.0
Agent: Q-Learning (Test), Solve rate: 100/100 (100.0%), First Solve: 1, First Solve Memory: 78, Steps: Avg 7.0, Min 7.0, Max 7.0
Agent: Go-Explore-HR, Solve rate: 28/100 (28.0%), First Solve: 33, First Solve Memory: 27, Steps: Avg 20.0, Min 9.0, Max 31.0
Agent: Go-Explore GDIR-HR, Solve rate: 25/100 (25.0%), First Solve: 45, First Solve Memory: 27, Steps: Avg 22.0, Min 13.0, Max 31.0
Agent: Go-Explore-Count-HR, Solve rate: 100/100 (100.0%), First Solve: 1, First Solve Memory: 15, Step

In [32]:
env = HanoiEnv(numplates = 7)

In [33]:
for agent in [RandomAgent, TDAgent, QAgent]:
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = 2**(env.numplates+2), verbose = False)
    
    # for QAgent and TDAgent, we include the final number of solves with deterministic transition
    if agent == QAgent or agent == TDAgent:
        MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = 2**(env.numplates+2), verbose = False, eps = 0)
    
for agent in [GoExplore, GoExploreCount, CountAgent]:
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = 2**(env.numplates+2), verbose = False, intrinsic_fn = None)

    # for Intrinsic Agent, we also do the run with intrinsic motivation
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = 2**(env.numplates+2), verbose = False, intrinsic_fn = Disk)

Agent: Random, No solves at all, First Solve Memory: -, Total Memory: 0
Agent: TD-Learning (Train), No solves at all, First Solve Memory: 0, Total Memory: 259
Agent: TD-Learning (Test), No solves at all, First Solve Memory: 0, Total Memory: 259
Agent: Q-Learning (Train), No solves at all, First Solve Memory: 0, Total Memory: 764
Agent: Q-Learning (Test), No solves at all, First Solve Memory: 0, Total Memory: 764
Agent: Go-Explore-HR, No solves at all, First Solve Memory: 0, Total Memory: 269
Agent: Go-Explore GDIR-HR, No solves at all, First Solve Memory: 0, Total Memory: 269
Agent: Go-Explore-Count-HR, No solves at all, First Solve Memory: 0, Total Memory: 919
Agent: Go-Explore-Count GDIR-HR, No solves at all, First Solve Memory: 0, Total Memory: 919
Agent: Explore-Count-HR, Solve rate: 82/100 (82.0%), First Solve: 19, First Solve Memory: 1660, Steps: Avg 398.2, Min 397.0, Max 465.0
Agent: Explore-Count GDIR-HR, Solve rate: 91/100 (91.0%), First Solve: 10, First Solve Memory: 1095, St

In [34]:
env = HanoiEnv(numplates = 8)

In [35]:
for agent in [RandomAgent, TDAgent, QAgent]:
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 200, env = copy.deepcopy(env), agent = agent, maxsteps = 2**(env.numplates+2), verbose = False)
    
    # for QAgent and TDAgent, we include the final number of solves with deterministic transition
    if agent == QAgent or agent == TDAgent:
        MultiGame(numtries = 200, env = copy.deepcopy(env), agent = agent, maxsteps = 2**(env.numplates+2), verbose = False, eps = 0)
    
for agent in [GoExplore, GoExploreCount, CountAgent]:
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 200, env = copy.deepcopy(env), agent = agent, maxsteps = 2**(env.numplates+2), verbose = False, intrinsic_fn = None)

    # for Intrinsic Agent, we also do the run with intrinsic motivation
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 200, env = copy.deepcopy(env), agent = agent, maxsteps = 2**(env.numplates+2), verbose = False, intrinsic_fn = Disk)

Agent: Random, No solves at all, First Solve Memory: -, Total Memory: 0
Agent: TD-Learning (Train), No solves at all, First Solve Memory: 0, Total Memory: 536
Agent: TD-Learning (Test), No solves at all, First Solve Memory: 0, Total Memory: 536
Agent: Q-Learning (Train), No solves at all, First Solve Memory: 0, Total Memory: 1541
Agent: Q-Learning (Test), No solves at all, First Solve Memory: 0, Total Memory: 1541
Agent: Go-Explore-HR, No solves at all, First Solve Memory: 0, Total Memory: 502
Agent: Go-Explore GDIR-HR, No solves at all, First Solve Memory: 0, Total Memory: 479
Agent: Go-Explore-Count-HR, No solves at all, First Solve Memory: 0, Total Memory: 1837
Agent: Go-Explore-Count GDIR-HR, No solves at all, First Solve Memory: 0, Total Memory: 1837
Agent: Explore-Count-HR, Solve rate: 49/200 (24.5%), First Solve: 152, First Solve Memory: 5235, Steps: Avg 846.4, Min 844.0, Max 961.0
Agent: Explore-Count GDIR-HR, Solve rate: 183/200 (91.5%), First Solve: 18, First Solve Memory: 32

# Discrete Environment 4: Narrowest Action Pathway - Game of Nim with Perfect Opponent
- There are N matchsticks
- Each turn, players rotate to remove 1-X matchsticks
- Player to remove the last matchstick wins

In [36]:
class NimEnv:
    def __init__(self, nummatches = 21, nummoves = 3, deterministic = True):
        self.nummatches = nummatches
        self.nummoves = nummoves
        self.numsteps = 0
        self.reward = 0
        self.deterministic = deterministic
        self.afterstate = nummatches
        self.done = False
            
        # some variables to reset the environment
        self.startmatches = self.nummatches
        self.afterstate = nummatches
            
    def reset(self):
        self.nummatches = self.startmatches
        self.reward = 0
        self.done = False
        self.numsteps = 0
        
    # returns state representation
    def staterep(self):
        return str([self.nummatches, self.nummoves])
            
    # gets a valid move
    def getvalidmoves(self):
        validmoves = list(range(1, min(self.nummatches, self.nummoves)+1))
        # give a valid move just to help with the learning process
        if validmoves == []: validmoves = [0]
        return validmoves
    
    def step(self, move):
        if self.done:
            return
        
        validmoves = self.getvalidmoves()
        # randomly sample a move if not in validmoves
        if move not in validmoves:
            move = validmoves[np.random.randint(len(validmoves))]
        self.numsteps += 1
        
        # do your move
        self.nummatches -= move
        
        # get the afterstate
        self.afterstate = self.nummatches
        
        if self.nummatches == 0:
            self.done = True
            self.reward = 1
            return
        
        # do the perfect player's move
        # choose randomly if already at perfect number
        if self.nummatches % (self.nummoves+1) == 0:
            if self.deterministic:
                self.nummatches -= 1
            else:
                self.nummatches -= self.sample()
            
        # else make it perfect number
        else:
            self.nummatches -= self.nummatches % (self.nummoves+1)
            
        if self.nummatches == 0:
            # failure state just for accounting purposes
            self.nummatches = -1
            self.done = True
            return
    
    def sample(self):
        return np.random.choice(self.getvalidmoves())
    
    def print(self):
        print(self.nummatches, self.nummoves)

In [37]:
''' Gets the answer to remove matchsticks by modulo '''
def Modulo(env):
    return -(env.afterstate % (env.nummoves+1))/env.nummoves

In [38]:
''' Gives a distance based on number of matches away from the goal '''
def CountMatches(env):
    return -env.afterstate/env.startmatches

In [39]:
env = NimEnv(nummatches = 11, nummoves = 3, deterministic = True)

In [40]:
for agent in [RandomAgent, TDAgent, QAgent]:
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = env.nummatches, verbose = False)
    
    # for QAgent and TDAgent, we include the final number of solves with deterministic transition
    if agent == QAgent or agent == TDAgent:
        MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = env.nummatches, verbose = False, eps = 0)
    
for agent in [GoExplore, GoExploreCount, CountAgent]:
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = env.nummatches, verbose = False, intrinsic_fn = None)

    # for Intrinsic Agent, we also do the run with intrinsic motivation
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = env.nummatches, verbose = False, intrinsic_fn = CountMatches)
    

Agent: Random, Solve rate: 4/100 (4.0%), First Solve: 71, First Solve Memory: -, Steps: Avg 3.0, Min 3.0, Max 3.0
Agent: TD-Learning (Train), Solve rate: 5/100 (5.0%), First Solve: 27, First Solve Memory: 7, Steps: Avg 3.0, Min 3.0, Max 3.0
Agent: TD-Learning (Test), Solve rate: 100/100 (100.0%), First Solve: 1, First Solve Memory: 7, Steps: Avg 3.0, Min 3.0, Max 3.0
Agent: Q-Learning (Train), Solve rate: 5/100 (5.0%), First Solve: 27, First Solve Memory: 17, Steps: Avg 3.0, Min 3.0, Max 3.0
Agent: Q-Learning (Test), Solve rate: 100/100 (100.0%), First Solve: 1, First Solve Memory: 17, Steps: Avg 3.0, Min 3.0, Max 3.0
Agent: Go-Explore-HR, Solve rate: 15/100 (15.0%), First Solve: 4, First Solve Memory: 7, Steps: Avg 3.0, Min 3.0, Max 3.0
Agent: Go-Explore GDIR-HR, Solve rate: 17/100 (17.0%), First Solve: 4, First Solve Memory: 7, Steps: Avg 3.0, Min 3.0, Max 3.0
Agent: Go-Explore-Count-HR, Solve rate: 98/100 (98.0%), First Solve: 3, First Solve Memory: 7, Steps: Avg 3.0, Min 3.0, Max 3

In [41]:
env = NimEnv(nummatches = 21, nummoves = 3, deterministic = True)

In [42]:
for agent in [RandomAgent, TDAgent, QAgent]:
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = env.nummatches, verbose = False)
    
    # for QAgent and TDAgent, we include the final number of solves with deterministic transition
    if agent == QAgent or agent == TDAgent:
        MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = env.nummatches, verbose = False, eps = 0)
    
for agent in [GoExplore, GoExploreCount, CountAgent]:
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = env.nummatches, verbose = False, intrinsic_fn = None)

    # for Intrinsic Agent, we also do the run with intrinsic motivation
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = env.nummatches, verbose = False, intrinsic_fn = CountMatches)
    

Agent: Random, No solves at all, First Solve Memory: -, Total Memory: 0
Agent: TD-Learning (Train), No solves at all, First Solve Memory: 0, Total Memory: 11
Agent: TD-Learning (Test), No solves at all, First Solve Memory: 0, Total Memory: 11
Agent: Q-Learning (Train), No solves at all, First Solve Memory: 0, Total Memory: 28
Agent: Q-Learning (Test), No solves at all, First Solve Memory: 0, Total Memory: 28
Agent: Go-Explore-HR, Solve rate: 16/100 (16.0%), First Solve: 5, First Solve Memory: 12, Steps: Avg 6.0, Min 6.0, Max 6.0
Agent: Go-Explore GDIR-HR, Solve rate: 7/100 (7.0%), First Solve: 5, First Solve Memory: 12, Steps: Avg 6.0, Min 6.0, Max 6.0
Agent: Go-Explore-Count-HR, Solve rate: 98/100 (98.0%), First Solve: 3, First Solve Memory: 12, Steps: Avg 6.0, Min 6.0, Max 6.0
Agent: Go-Explore-Count GDIR-HR, Solve rate: 98/100 (98.0%), First Solve: 3, First Solve Memory: 12, Steps: Avg 6.0, Min 6.0, Max 6.0
Agent: Explore-Count-HR, Solve rate: 98/100 (98.0%), First Solve: 3, First S

In [43]:
env = NimEnv(nummatches = 1001, nummoves = 3, deterministic = True)

In [44]:
for agent in [RandomAgent, TDAgent, QAgent]:
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = env.nummatches, verbose = False)
    
    # for QAgent and TDAgent, we include the final number of solves with deterministic transition
    if agent == QAgent or agent == TDAgent:
        MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = env.nummatches, verbose = False, eps = 0)
    
for agent in [GoExplore, GoExploreCount, CountAgent]:
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = env.nummatches, verbose = False, intrinsic_fn = None)

    # for Intrinsic Agent, we also do the run with intrinsic motivation
    memory = defaultdict(lambda: 0)
    MultiGame(numtries = 100, env = copy.deepcopy(env), agent = agent, maxsteps = env.nummatches, verbose = False, intrinsic_fn = CountMatches)

Agent: Random, No solves at all, First Solve Memory: -, Total Memory: 0
Agent: TD-Learning (Train), No solves at all, First Solve Memory: 0, Total Memory: 256
Agent: TD-Learning (Test), No solves at all, First Solve Memory: 0, Total Memory: 256
Agent: Q-Learning (Train), No solves at all, First Solve Memory: 0, Total Memory: 763
Agent: Q-Learning (Test), No solves at all, First Solve Memory: 0, Total Memory: 763
Agent: Go-Explore-HR, No solves at all, First Solve Memory: 0, Total Memory: 253
Agent: Go-Explore GDIR-HR, No solves at all, First Solve Memory: 0, Total Memory: 253
Agent: Go-Explore-Count-HR, No solves at all, First Solve Memory: 0, Total Memory: 254
Agent: Go-Explore-Count GDIR-HR, No solves at all, First Solve Memory: 0, Total Memory: 252
Agent: Explore-Count-HR, Solve rate: 98/100 (98.0%), First Solve: 3, First Solve Memory: 502, Steps: Avg 251.0, Min 251.0, Max 251.0
Agent: Explore-Count GDIR-HR, Solve rate: 99/100 (99.0%), First Solve: 2, First Solve Memory: 502, Steps: